# Outlays for Discretionary Programs

Source: https://www.whitehouse.gov/omb/historical-tables/

Table 8.8 - Outlays for Discretionary Programs in Constant (FY 2009) Dollars: 1962–2023 

Values in billions of dollars

In [44]:
import pandas as pd

In [98]:
def load_data(url):
    # Parse excel file from url
    df = pd.read_excel(url, skiprows=2)
    df = df.dropna()
    df = df.replace("..........", 0)
    df.index = df['Category and Program']
    del df['Category and Program']
    
    df.columns = [i.replace(" estimate","") for i in df.columns]

    # Only include actual years, not future year estimates
    years_to_include = [str(i) for i in range(1962,2024)]
    df = df[years_to_include]
    return df

In [99]:
def parse_data(df):
    
    # Define categories
    military = df.loc['Total national defense']
    international_affairs = df.loc['International affairs']
    science = df.loc['Total general science, space and technology']
    energy_and_environment = df.loc[['Energy', 'Natural resources and environment']].sum()
    agriculture = df.loc['Agriculture']
    housing_and_community = df.loc[['Commerce and housing credit', 'Community and regional development',
                                    'Total income security']].sum()
    education = df.loc['Education']
    transportation = df.loc['Total transportation']
    social_security_unemployment_labor = df.loc[['Training, employment and social services',
                                                 'Social security']].sum()
    veterans_benefits = df.loc['Veterans benefits and services']
    government = df.loc[['Administration of justice', 'General government']].sum()
    health_and_medicare = df.loc[['Health', 'Medicare']].sum()
    
    # Name categories
    military.name = 'Military'
    international_affairs.name = 'International Affairs'
    science.name = 'Science'
    energy_and_environment.name = 'Energy and Environment'
    agriculture.name = 'Agriculture'
    housing_and_community.name = 'Housing and Community'
    education.name = 'Education'
    transportation.name = 'Transportation'
    social_security_unemployment_labor.name = 'Soc. Security, Unemployment & Labor'
    veterans_benefits.name = 'Veterans Benefits'
    government.name = 'Government'
    health_and_medicare.name = 'Health and Medicare'
    
    # Sort categories in descending order (cumulative)
    unsorted_categories = [military, international_affairs, science,
                      energy_and_environment, agriculture, housing_and_community,
                      education, transportation, social_security_unemployment_labor,
                      veterans_benefits, government, health_and_medicare]
    cumulative_sums = [i.sum() for i in unsorted_categories]
    sorted_categories = [x for _, x in sorted(zip(cumulative_sums,unsorted_categories), reverse=True)]
    
    # Create dataframe
    output = pd.DataFrame(sorted_categories)
    output = output.transpose()
    return output

In [100]:
def build_pie_chart_data(df, frames_per_year):
    pie = df.copy()
    pie['Total'] = pie.sum(axis=1)
    
    for i in pie.columns:
        new_col_name = str(i + "_prev")
        pie[new_col_name] = pie[i].shift(1).fillna(0)
        
    pie['year'] = pie.index.astype(float)
    min_year = min(pie['year'])
    pie['start_frame'] = pie['year'].astype(float) * frames_per_year - (min_year * frames_per_year)
    pie['stop_frame'] = pie['start_frame'] + frames_per_year
    
    pie = pie.fillna(0)
    pie = pie.clip(lower=0)
    
    return pie

In [101]:
url = "https://www.whitehouse.gov/wp-content/uploads/2018/02/hist08z8-fy2019.xlsx"

In [102]:
raw_data = load_data(url)

In [103]:
df = parse_data(raw_data)

In [104]:
pie_df = build_pie_chart_data(df, frames_per_year = 60)

In [106]:
pie_df.to_csv("../data/discretionary_outlays.csv")

In [105]:
pie_df

,Military,Housing and Community,Transportation,Education,Government,Energy and Environment,Health and Medicare,International Affairs,Veterans Benefits,"Soc. Security, Unemployment & Labor",...,Health and Medicare_prev,International Affairs_prev,Veterans Benefits_prev,"Soc. Security, Unemployment & Labor_prev",Science_prev,Agriculture_prev,Total_prev,year,start_frame,stop_frame
1962,400.5,17.3,10.9,5.2,12.9,22.5,7.5,41.9,7.4,4.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1962.0,0.0,60.0
1963,392.4,17.0,10.6,5.8,13.1,22.8,8.5,37.6,7.6,4.9,...,7.5,41.9,7.4,4.8,13.1,2.9,546.9,1963.0,60.0,120.0
1964,396.5,18.4,10.3,6.0,14.7,23.0,10.1,31.5,7.9,5.8,...,8.5,37.6,7.6,4.9,21.9,3.4,545.6,1964.0,120.0,180.0
1965,368.1,21.7,10.8,7.3,14.0,24.2,9.2,30.9,8.0,8.0,...,10.1,31.5,7.9,5.8,33.5,3.5,561.2,1965.0,180.0,240.0
1966,406.6,22.9,10.7,16.0,14.6,24.9,10.8,32.3,8.1,15.2,...,9.2,30.9,8.0,8.0,38.2,3.6,544.0,1966.0,240.0,300.0
1967,480.6,25.3,11.8,23.8,15.8,26.2,14.2,33.7,8.4,19.4,...,10.8,32.3,8.1,15.2,42.9,4.0,609.0,1967.0,300.0,360.0
1968,523.9,26.3,12.6,26.6,15.8,27.5,16.7,30.0,8.6,23.6,...,14.2,33.7,8.4,19.4,39.2,4.4,702.8,1968.0,360.0,420.0
1969,499.2,20.4,13.7,24.0,15.1,25.6,17.4,23.4,8.8,22.9,...,16.7,30.0,8.6,23.6,34.0,4.9,750.5,1969.0,420.0,480.0
1970,470.2,31.0,14.2,26.7,17.1,24.6,18.4,21.4,9.6,23.0,...,17.4,23.4,8.8,22.9,28.7,4.6,703.8,1970.0,480.0,540.0
1971,426.4,36.0,19.7,27.9,18.3,26.3,18.6,18.6,10.3,24.9,...,18.4,21.4,9.6,23.0,24.3,4.7,685.2,1971.0,540.0,600.0
